# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [1]:
from sagemaker import get_execution_role

In [2]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20230712T200237


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../images/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../images/inline-policy.jpg)

In [14]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../images/trust-relationship.jpg)

In [15]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Install python SDK for bedrock

In [3]:
#!wget https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip
!unzip bedrock-python-sdk.zip -d bedrock-sdk
!rm -rf bedrock-python-sdk.zip

--2023-08-08 05:00:04--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 18.160.0.129, 18.160.0.61, 18.160.0.63, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|18.160.0.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251046205 (239M) [application/zip]
Saving to: ‘bedrock-python-sdk.zip’

100%[======================================>] 251,046,205 94.3MB/s   in 2.5s   

2023-08-08 05:00:06 (94.3 MB/s) - ‘bedrock-python-sdk.zip’ saved [251046205/251046205]

Archive:  bedrock-python-sdk.zip
   creating: bedrock-sdk/reviews/
  inflating: bedrock-sdk/awscli-1.29.21.tar.gz  
  inflating: bedrock-sdk/AWSCLI32PY3.msi  
  inflating: bedrock-sdk/manifest.json  
  inflating: bedrock-sdk/AWSCLISetup.exe  
  inflating: bedrock-sdk/.functional  
  inflating: bedrock-sdk/awscli-bundle.zip  
  inflating: bedrock-sdk/boto3-1.28.21-py3-non

In [4]:
install_needed = True

In [5]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-*-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.4/857.4 kB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for PyYAML~=6.0 from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 85.6 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.175.0-py2.py3-none-any.whl size=1165569 sha256=26a9da6f6366fa5cfda28524b2

## 3. Check setting
아래 메시지가 뜨면 정상 <BR>
![nn](../images/check.png)


In [1]:
import boto3
import langchain

In [2]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.256
boto3 version check: 1.28.21


In [3]:
is_internal_use = True # 내부 직원 용
# is_internal_use = False # 고객 용

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [4]:
bedrock_region = "us-east-1" 

In [10]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

In [11]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [12]:
output_text = bedrock_client.list_foundation_models()
output_text 

{'ResponseMetadata': {'RequestId': '8d3f0bf4-fa07-41ea-b13a-cb1a62d24746',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 08 Aug 2023 05:02:29 GMT',
   'content-type': 'application/json',
   'content-length': '972',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8d3f0bf4-fa07-41ea-b13a-cb1a62d24746'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in